In [2]:
import pandas as pd
import numpy as np
import pymongo
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [52]:
ds = pd.read_excel('DBRaw.xlsx')

In [5]:
client = pymongo.MongoClient(os.getenv("URI"))

In [14]:
db = client["Nefro"]
collection = db["exams"]
collection.find_one({"codigo": "33400135"})["exams"]

[{'name': 'Índice de Massa Corporal',
  'data': [{'x': datetime.datetime(2014, 9, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2014, 10, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2014, 11, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2014, 12, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 1, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 2, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 3, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 4, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 5, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 6, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 7, 1, 0, 0), 'y': 19.5},
   {'x': datetime.datetime(2015, 8, 1, 0, 0), 'y': 19.3},
   {'x': datetime.datetime(2015, 9, 1, 0, 0), 'y': 19.3},
   {'x': datetime.datetime(2015, 10, 1, 0, 0), 'y': 19.3},
   {'x': datetime.datetime(2015, 11, 1, 0, 0), 'y': 19.3}]},
 {'name': 'Anti-HBs (Titulação)',
  'data': [{'x': datetime.datetime(2014, 9,

In [7]:
pacients_exams = collection.find()

In [ ]:
# collection.count_documents({})
pacients_exams[8]["codigo"]
pacient = ds[ds["Código"] == pacients_exams[0]["codigo"]]
# columns = ds.drop(columns=["Código", "Mês/ano","Unnamed: 0","Sexo","Idade","Desfecho"]).columns

columns = ['Índice de Massa Corporal', 'Anti-HBs (Titulação)', 'Anti-HBs',
       'HBs Ag', 'Anti-HIV', 'Taxa de Redução da Uréia', 'K+', 'Na+',
       'Hb do mês atual', 'Ferritina', 'Saturação de Transferrina',
       'Alb.\n(Verde bromocresol)', 'Proteina Total', 'Colesterol Total',
       'Ca Total do mês atual', 'P', 'Último resultado do PTHi',
       'Fosfatase Alcalina Total']



export = []
for c in columns:
    aux = {}
    aux["name"] = c
    res = ds.loc[(ds[c].notna()) & (ds["Código"] == pacients_exams[0]["codigo"]),[c,"Mês/ano"]]
    print(res)
    lista_objetos = [{'x': row['Mês/ano'], 'y': row[c]} for _, row in res.iterrows()]
    aux["data"] = lista_objetos

    export.append(aux)

print(export)

for e in export:
    print(len(e["data"]))


[{'name': 'Índice de Massa Corporal', 'data': [{'x': Timestamp('2014-09-01 00:00:00'), 'y': 23.6}, {'x': Timestamp('2014-10-01 00:00:00'), 'y': 23.3}, {'x': Timestamp('2014-11-01 00:00:00'), 'y': 23.1}, {'x': Timestamp('2014-12-01 00:00:00'), 'y': 23.1}, {'x': Timestamp('2015-01-01 00:00:00'), 'y': 22.4}, {'x': Timestamp('2015-02-01 00:00:00'), 'y': 21.7}, {'x': Timestamp('2015-03-01 00:00:00'), 'y': 21.7}, {'x': Timestamp('2015-04-01 00:00:00'), 'y': 21.7}, {'x': Timestamp('2015-05-01 00:00:00'), 'y': 21.5}, {'x': Timestamp('2015-06-01 00:00:00'), 'y': 21.5}, {'x': Timestamp('2015-07-01 00:00:00'), 'y': 21.5}, {'x': Timestamp('2015-08-01 00:00:00'), 'y': 21.5}]}, {'name': 'Anti-HBs (Titulação)', 'data': [{'x': Timestamp('2014-09-01 00:00:00'), 'y': 3.0}, {'x': Timestamp('2014-10-01 00:00:00'), 'y': 3.0}, {'x': Timestamp('2014-11-01 00:00:00'), 'y': 3.0}, {'x': Timestamp('2014-12-01 00:00:00'), 'y': 3.0}, {'x': Timestamp('2015-01-01 00:00:00'), 'y': 3.0}, {'x': Timestamp('2015-02-01 00